In [21]:
import os
from flask import Flask, request, jsonify
from flask_cors import CORS 
# import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
import pymysql
import numpy as np
import datetime
import json
from dotenv import load_dotenv

In [25]:
app = Flask(__name__)
app.json.sort_keys = False
CORS(app) 

db = pymysql.connect(
    host=os.getenv('DATABASE_HOST'),
    user=os.getenv('DATABASE_USER'),
    password=os.getenv('DATABASE_PASSWORD'),
    db=os.getenv('DATABASE_DB'),
    connect_timeout=8800,
    cursorclass=pymysql.cursors.DictCursor
)

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kimbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [114]:
sql_query= """
            SELECT 
                keyword
            FROM 
                article 
            # WHERE status = 1
           """
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute(sql_query)
data = cursor.fetchall()

print(data)
keywords = [row['keyword'] for row in data]

[{'keyword': 'call center representatives, labor productivity, and employment'}, {'keyword': 'social isolation, work satisfaction, stress, work productivity'}, {'keyword': 'electronic wallet, accuracy of payment, on-time delivery, accessibility and efficiency, security and reliability'}, {'keyword': 'Safety of life at sea program (SOLAS), seafarers, MARITIME accidents, risk reduction management program'}, {'keyword': 'access, faculty portal, features, usability, Universidad de manila'}, {'keyword': 'food safety, sanitation, street food, food handling, consumer'}, {'keyword': ''}, {'keyword': 'speaking, video recording, oral communication, integration, online learning'}, {'keyword': 'challenges, online learning, Covid-19 pandemic, laboratory teachers, implementation.'}, {'keyword': 'challenges, online learning, Covid-19 pandemic, laboratory'}, {'keyword': 'challenges, online learning, Covid-19 pandemic, laboratory teachers, implementation'}, {'keyword': 'curriculum enhancement, subject 

In [108]:
keywords

['call center representatives, labor productivity, and employment',
 'social isolation, work satisfaction, stress, work productivity',
 'electronic wallet, accuracy of payment, on-time delivery, accessibility and efficiency, security and reliability',
 'Safety of life at sea program (SOLAS), seafarers, MARITIME accidents, risk reduction management program',
 'access, faculty portal, features, usability, Universidad de manila',
 'food safety, sanitation, street food, food handling, consumer',
 '',
 'speaking, video recording, oral communication, integration, online learning',
 'challenges, online learning, Covid-19 pandemic, laboratory teachers, implementation.',
 'challenges, online learning, Covid-19 pandemic, laboratory',
 'challenges, online learning, Covid-19 pandemic, laboratory teachers, implementation',
 'curriculum enhancement, subject understanding, personal qualities, skill, employability',
 'academic performance, aviation, PhilSCA',
 'education, course preference, students',

In [140]:
extractedKeywords = []
for keys in keywords:
    keys = keys.replace(' ','_').split(',_')
    for key in keys:
        temp = key.lower().strip()
        temp = '_'.join([word for word in temp.split('_') if word not in stop_words])
        temp = ''.join([letter for letter in temp if letter.isalpha() or letter=='_' ])
        if len(temp) != 0:
            key = temp.replace('_',' ') 
            if key.strip() not in extractedKeywords:
                extractedKeywords.append(key)
        
        
    

In [101]:
# for n, name in enumerate(extractedKeywords):
#     temp = name.lower().split(" ")
#     temp = [''.join([letter for letter in word if letter.isalpha()]) for word in temp]
#     temp = [word for word in temp if word not in stop_words]
#     temp = ' '.join(temp)
#     extractedKeywords[n] = temp

In [141]:
extractedKeywords

['call center representatives',
 'labor productivity',
 'employment',
 'social isolation',
 'work satisfaction',
 'stress',
 'work productivity',
 'electronic wallet',
 'accuracy payment',
 'ontime delivery',
 'accessibility efficiency',
 'security reliability',
 'safety life sea program solas',
 'seafarers',
 'maritime accidents',
 'risk reduction management program',
 'access',
 'faculty portal',
 'features',
 'usability',
 'universidad de manila',
 'food safety',
 'sanitation',
 'street food',
 'food handling',
 'consumer',
 'speaking',
 'video recording',
 'oral communication',
 'integration',
 'online learning',
 'challenges',
 'covid pandemic',
 'laboratory teachers',
 'implementation',
 'laboratory',
 'curriculum enhancement',
 'subject understanding',
 'personal qualities',
 'skill',
 'employability',
 'academic performance',
 'aviation',
 'philsca',
 'education',
 'course preference',
 'students',
 'modular distance learning',
 'stakeholders involvement',
 'remote leaming enga

In [139]:
with open('extractedKeywords.json', 'w') as file:
    json.dump(extractedKeywords, file)